In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [1]:
import pandas as pd
df = pd.read_csv('50_Startups.csv')
df.head()

,RND,ADMIN,MKT,STATE,PROFIT
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RND     50 non-null     float64
 1   ADMIN   50 non-null     float64
 2   MKT     50 non-null     float64
 3   STATE   50 non-null     object 
 4   PROFIT  50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [3]:
df.isna().sum()

RND       0
ADMIN     0
MKT       0
STATE     0
PROFIT    0
dtype: int64

In [4]:
D = df.duplicated().sum()
D

0

In [5]:
X = df.drop(columns=['STATE', 'PROFIT'])
X.head()

,RND,ADMIN,MKT
0,165349.20,136897.80,471784.10
1,162597.70,151377.59,443898.53
2,153441.51,101145.55,407934.54
3,144372.41,118671.85,383199.62
4,142107.34,91391.77,366168.42


In [6]:
Y = df[['PROFIT']]
Y.head()

,PROFIT
0,192261.83
1,191792.06
2,191050.39
3,182901.99
4,166187.94


In [12]:
from sklearn.model_selection import train_test_split, cross_val_score

In [13]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42)


In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xtest_scaled = scaler.transform(xtest)

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [8]:
dct = {'Linear Regression' : LinearRegression(),
       'Decision Tree' : DecisionTreeRegressor(),
       'Random Forest': RandomForestRegressor(),
       'Gradient Boost': GradientBoostingRegressor(),
       'KNN': KNeighborsRegressor(),
       'SVM': SVR()}

In [9]:
dct.items()

dict_items([('Linear Regression', LinearRegression()), ('Decision Tree', DecisionTreeRegressor()), ('Random Forest', RandomForestRegressor()), ('Gradient Boost', GradientBoostingRegressor()), ('KNN', KNeighborsRegressor()), ('SVM', SVR())])

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [20]:
import numpy as np

In [21]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Cross-validation (using R^2 as the scoring metric)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    
    # Fit model to training data
    model.fit(X_train, y_train)
    
    # Predictions on the test set
    y_pred = model.predict(X_test)
    
    # Compute evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    # Return results
    return {
        'model': model.__class__.__name__,
        'mean_cv_r2': np.mean(cv_scores),
        'test_r2': r2,
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
    }


In [30]:
results = []

for name, model in dct.items():
    result = evaluate_model(model, xtrain_scaled, ytrain, xtest_scaled, ytest)
    results.append(result)

# 7. Convert results into a DataFrame for better visualization
results_df = pd.DataFrame(results)

# 8. Sort models based on RMSE (for example, the lower the better)
sorted_results = results_df.sort_values(by='rmse')

# 9. Print the best performing model and evaluation metrics
print("Best Performing Model Based on RMSE:")
print(sorted_results.iloc[0])

# Optionally, you can sort based on other metrics like R^2 or MAE.
print("\nAll Evaluation Results (sorted by RMSE):")
print(sorted_results)

c:\Users\Ashutosh\Downloads\Wakad Thu Fri 6 8\repository\venv\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ashutosh\Downloads\Wakad Thu Fri 6 8\repository\venv\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ashutosh\Downloads\Wakad Thu Fri 6 8\repository\venv\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ashutosh\Downloads\Wakad Thu Fri 6 8\repository\venv\Lib\site-packages\sklea

Best Performing Model Based on RMSE:
model         LinearRegression
mean_cv_r2            0.940059
test_r2               0.900065
mae                6979.152252
mse            80926321.222952
rmse               8995.905803
Name: 0, dtype: object

All Evaluation Results (sorted by RMSE):
                       model  mean_cv_r2   test_r2           mae  \
0           LinearRegression    0.940059  0.900065   6979.152252   
2      RandomForestRegressor    0.937077  0.895261   6209.813790   
3  GradientBoostingRegressor    0.926861  0.895218   8573.489585   
1      DecisionTreeRegressor    0.916409  0.856477   8163.252000   
4        KNeighborsRegressor    0.835992  0.524487  16154.798800   
5                        SVR   -0.060964 -0.180081  22846.725400   

            mse          rmse  
0  8.092632e+07   8995.905803  
2  8.481706e+07   9209.617665  
3  8.485132e+07   9211.477638  
1  1.162235e+08  10780.699336  
4  3.850669e+08  19623.122367  
5  9.556204e+08  30913.109958  


c:\Users\Ashutosh\Downloads\Wakad Thu Fri 6 8\repository\venv\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Ashutosh\Downloads\Wakad Thu Fri 6 8\repository\venv\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Ashutosh\Downloads\Wakad Thu Fri 6 8\repository\venv\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Ashutosh\Downloads\Wakad Thu Fri 6 8\repository\venv\Lib\site-packages\sklearn\uti

In [32]:
results_df = pd.DataFrame(results)

# 9. Sort models based on RMSE (the lower the better)
sorted_results = results_df.sort_values(by='rmse')

# 10. Print the best performing model and evaluation metrics
print("Best Performing Model Based on RMSE:")
print(sorted_results.iloc[0])

# Optionally, you can sort based on other metrics like R^2 or MAE.
print("\nAll Evaluation Results (sorted by RMSE):")
print(sorted_results)

# 11. Train the best model on the full dataset and make predictions
best_model_name = sorted_results.iloc[0]['model']
best_model = dct[best_model_name]
best_model.fit(xtrain_scaled, ytrain)

# Make predictions on the test set
y_pred_best = best_model.predict(xtest_scaled)

# Print predictions and actual values for comparison
print("\nPredictions from the Best Model:")
print(pd.DataFrame({'Actual': ytest, 'Predicted': y_pred_best}))

# Optionally, evaluate the final model on the test set
final_r2 = r2_score(ytest, y_pred_best)
final_mae = mean_absolute_error(ytest, y_pred_best)
final_rmse = np.sqrt(mean_squared_error(ytest, y_pred_best))

print(f"\nFinal Model Evaluation (Best Model - {best_model_name}):")
print(f"R²: {final_r2}")
print(f"MAE: {final_mae}")
print(f"RMSE: {final_rmse}")

Best Performing Model Based on RMSE:
model         LinearRegression
mean_cv_r2            0.940059
test_r2               0.900065
mae                6979.152252
mse            80926321.222952
rmse               8995.905803
Name: 0, dtype: object

All Evaluation Results (sorted by RMSE):
                       model  mean_cv_r2   test_r2           mae  \
0           LinearRegression    0.940059  0.900065   6979.152252   
2      RandomForestRegressor    0.937077  0.895261   6209.813790   
3  GradientBoostingRegressor    0.926861  0.895218   8573.489585   
1      DecisionTreeRegressor    0.916409  0.856477   8163.252000   
4        KNeighborsRegressor    0.835992  0.524487  16154.798800   
5                        SVR   -0.060964 -0.180081  22846.725400   

            mse          rmse  
0  8.092632e+07   8995.905803  
2  8.481706e+07   9209.617665  
3  8.485132e+07   9211.477638  
1  1.162235e+08  10780.699336  
4  3.850669e+08  19623.122367  
5  9.556204e+08  30913.109958  


KeyError: 'LinearRegression'